In [33]:
from kafka import KafkaConsumer, TopicPartition
import json
from transofrm import TRANSFORMER_LIST
from datetime import timedelta, datetime, timezone
from dateutil import parser
import json
import time
from influxdb_client import InfluxDBClient, Point, WriteOptions
from influxdb_client.client.write_api import SYNCHRONOUS

In [25]:
consumer = KafkaConsumer(
                        bootstrap_servers = 'rc1a-b5e65f36lm3an1d5.mdb.yandexcloud.net:9091',
                        client_id='9433_reader',
                        security_protocol='SASL_SSL',
                        ssl_cafile='CA.pem',
                        retry_backoff_ms=10,
                        auto_offset_reset='latest',
                        sasl_mechanism='SCRAM-SHA-512',
                        sasl_plain_username='9433_reader',
                        sasl_plain_password='eUIpgWu0PWTJaTrjhjQD3.hoyhntiK',
                        group_id='ЕВРАЗ 2.0 - Поток 7 - Бассейн данных',
                        value_deserializer = lambda m: json.loads(m.decode('utf-8')))

In [27]:
client = InfluxDBClient(
   url="http://localhost:8086",
   token="3qiRayG1_-FXr2VegUh7eQ5uFGeU65t8pvpCEMsZuswV04TZWjyLUHq4Gie7hIUJYbP6SzWlETrggQlCG1yQ4Q==",
   org="kafka-hack"
)
writer = client.write_api(write_options=SYNCHRONOUS)

In [ ]:
topic_partition = TopicPartition('zsmk-9433-dev-01', 0)
consumer.assign([topic_partition])
consumer.seek_to_beginning(topic_partition)
for msg in consumer:
    print(msg.value['moment'])
    point = Point("exhauster")
    point.tag("exhauster", "exhauster")
    point.time(parser.parse(msg.value["moment"]))
    for key, value in msg.value.items():
        point.field(key, value)
    print(point)
    writer.write(bucket="kafka-alldata", record=[point])